# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,
and responds with an explanation. This is a tool that you will be able to use yourself during the course!


In [1]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
import ipywidgets as widgets


In [2]:
!ollama pull llama3.2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


In [4]:
# set up environment

# Get the openai key

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-') and len(openai_api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()
# Get the ollama key using the llama model

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')



API key looks good so far


In [5]:
system_prompt = "You are a knowledgeable technical instructor who helps students understand \
complex concepts across a wide range of technical topics. Your expertise includes artificial]\
intelligence, machine learning, large language models (LLMs), and programming in languages \
such as Python, JavaScript, Java, and more. You also provide in-depth support for \
AI engineering questions and other advanced technical subjects."


In [6]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [7]:
# Helper to answer using selected model without duplicating logic

def answer_question(question_text: str, model_choice: str):
    if not question_text or not question_text.strip():
        display(Markdown("**Please enter a question.**"))
        return

    if model_choice == MODEL_GPT:
        stream = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question_text},
            ],
            stream=True,
        )
        response_text = ""
        handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response_text += chunk.choices[0].delta.content or ""
            response_text = response_text.replace("```", "").replace("markdown", "")
            update_display(Markdown(response_text), display_id=handle.display_id)
        return

    if model_choice == MODEL_LLAMA:
        response = ollama_via_openai.chat.completions.create(
            model=MODEL_LLAMA,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question_text},
            ],
        )
        result = response.choices[0].message.content
        display(Markdown(result))
        return

    display(Markdown(f"Unknown model: {model_choice}"))


In [8]:
# Get gpt-4o-mini to answer, with streaming
answer_question(question, MODEL_GPT)


Certainly! The code snippet you provided is a Python expression that uses a combination of a generator, a set comprehension, and a conditional filter. Let’s break it down step by step.

### Components of the Code

1. **`books`**: This is assumed to be an iterable, likely a list of dictionaries, where each dictionary represents a book with various attributes (like title, author, etc.).

2. **`book.get("author")`**: This is a method call that retrieves the value associated with the key `"author"` from each `book` dictionary. If the `"author"` key is not present, `book.get()` will return `None`.

3. **Set Comprehension**: The expression `{book.get("author") for book in books if book.get("author")}` creates a set of authors from the `books`. The `if book.get("author")` condition ensures that only books where an author is defined (non-`None`) are included in the set. This means the result will only contain unique author names, as sets do not allow duplicate entries.

4. **`yield from`**: This keyword is part of Python's generator function syntax. It allows you to yield all values from an iterable (in this case, the set of authors) one by one. 

### What the Code Does

Putting it all together, this code does the following:

- It generates a set of unique author names (`book.get("author")`) from the provided `books` list, but only for those books that have a defined author.
- By using `yield from`, the generator will yield each author name one at a time whenever the generator function is called.

### Example

For instance, if `books` is structured as follows:

python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": "Author 1"},  # Duplicate
    {"title": "Book D"},  # No author
    {"title": "Book E", "author": "Author 3"}
]


The code would execute as follows:

1. Create the set of authors: `{'Author 1', 'Author 2', 'Author 3'}`
2. Yield each author when the generator is iterated over.

### Why Use This Code?

- **Efficiency**: It collects unique authors, which can be useful in scenarios where you only want distinct entries, eliminating duplicates effortlessly since sets handle this inherently.
- **Memory Management**: Using `yield` allows for generating values on-the-fly, which can save memory when working with large datasets, as it does not require storing all the results in memory at once.
- **Readability and Conciseness**: The combination of comprehensions and generators makes the code both concise and expressive, which can improve readability and maintainability.

Overall, this code snippet is a Pythonic way to extract unique author names from a collection of book dictionaries while avoiding any unnecessary complexity.

In [9]:
# Get Llama 3.2 to answer
answer_question(question, MODEL_LLAMA)


This line of code is using a feature called "generator expressions" in Python, which is a concise way to create iterators.

Here's a breakdown of what the code does:

- `yield from`: This keyword is used when you want to delegate iteration over other items. It tells Python to yield values from another iterable.

- `{book.get("author") for book in books if book.get("author")}`: This part creates a generator expression, which generates values on the fly and returns an iterator object.

  - `for book in books`: This part iterates over each item (`book`) in the collection of `books`.

  - `if book.get("author")`: This condition filters out items that do not have an 'author' key. It uses the `.get()` method to safely retrieve the value, avoiding a `KeyError` if the key does not exist.

  - `book.get("author")`: This part of the expression retrieves the author from each book.

The overall effect is to produce an iterator over the author names of books that have authors. 

In other words, this line of code creates an iterator over the authors of all books in the collection where the author exists. 

This generator expression can be used to simplify a loop and make it more expressive, as we don't need to manually iterate over the items with explicit loops.

Here's how you could rewrite a simple loop using `yield from`:

Before:
```python
author_names = []
for book in books:
    if book.get("author"):
        author_names.append(book.get("author"))
```
After:
```python
yield from {book.get("author") for book in books if book.get("author")}
# this would yield the same result, without needing an explicit loop.
```

However, keep in mind that using `yield from` and generator expressions will stop once the inner iterable does. So, you might need to make sure that's the desired behavior for your use case.

Finally, note that `book.get("author")` is called twice here (once when defining the generator expression and again when yielding the results). In Python 3.x, this can be problematic due to a feature known as "surprise evaluation" of function arguments. It can lead to unpredictable behavior in older versions of Python or if you're reusing the same `book` object across multiple calls.

If you're able, consider defining the value twice separately and storing them individually. For instance:

```python
author_values = [book.get("author") for book in books if book.get("author")]
yield from author_values
```
However, this avoids the problem of surprise evaluation. If you really need to use a generator expression or `yield from`, using it as part of an outer loop might mitigate some issues by having explicit control of when the outer function is called with values. This way you don't necessarily see the inner values being evaluated twice unless you want them to be. 

Note however that sometimes things might sound "clever" but may still potentially create problems or unexpected execution flows without properly thinking what should be the flow in your given scenario

In [ ]:
# ipywidgets UI

model_dropdown = widgets.Dropdown(
    options=[("GPT", MODEL_GPT), ("LLaMA", MODEL_LLAMA)],
    value=MODEL_GPT,
    description="Model:",
)

question_area = widgets.Textarea(
    value="",
    placeholder="Ask your technical question here...",
    description="Question:",
    layout=widgets.Layout(width='100%', height='120px')
)

ask_button = widgets.Button(description="Ask", button_style="primary")
output = widgets.Output()


def on_ask_clicked(_):
    output.clear_output()
    with output:
        answer_question(question_area.value, model_dropdown.value)


ask_button.on_click(on_ask_clicked)

ui = widgets.VBox([
    model_dropdown,
    question_area,
    ask_button,
    output,
])

ui
